In [1]:
import os
import numpy as np
from netCDF4 import Dataset
import matplotlib.pyplot as plt
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.crs as ccrs
import cartopy.feature as cfeature

This example will guide you through visualizing the aerosol optical thickness from Himawari-8 products.

You can find [here](../doc/2018_A_Yamashita.pdf) detailed description about this product.

In [2]:
projectDir = os.path.dirname(os.path.abspath(os.curdir))
latRange = [0, 55]   # latitude range
lonRange = [100, 160]   # longitude range
axLatRange = latRange
axLonRange = lonRange
CLP_file = 'NC_H08_20200219_0400_L2ARP021_FLDK.02401_02401.nc'   # Aerosol Property file
product = 'AOT'   # product
vmin = 0   # minimun AOT for display
vmax = 1   # maximum AOT for display

The product was saved in [netCDF4](https://www.unidata.ucar.edu/software/netcdf/) file. 

Below is the metadata for the AOT:
```text
short AOT(latitude=2401, longitude=2401);
  :long_name = "Aerosol optical thickness";
  :units = "Dimensionless";
  :valid_min = 0S; // short
  :valid_max = 25000S; // short
  :scale_factor = 2.0E-4f; // float
  :add_offset = 0.0f; // float
  :missing_value = -32768S; // short
  :_ChunkSizes = 1201U, 1201U; // uint
```

In [3]:
fd = Dataset(os.path.join(projectDir, 'data', CLP_file), 'r')

fd.variables.keys()
lat = fd.variables['latitude'][:]
lon = fd.variables['longitude'][:]

mask_lat = np.logical_and(lat >= latRange[0], lat <= latRange[1])
mask_lon = np.logical_and(lon >= lonRange[0], lon <= lonRange[1])

dataRect = fd.variables[product][:, mask_lon][mask_lat]

latRect = lat[mask_lat]
lonRect = lon[mask_lon]

In [4]:
# Load the border data, CN-border-La.dat is downloaded from
# https://gmt-china.org/data/CN-border-La.dat
with open(os.path.join(projectDir, 'include', 'CN-border-La.dat')) as src:
    context = src.read()
    blocks = [cnt for cnt in context.split('>') if len(cnt) > 0]
    borders = [np.fromstring(block, dtype=float, sep=' ') for block in blocks]

In [5]:
# Set figure size
fig = plt.figure(figsize=[10, 8])

# Set projection and plot the main figure
ax = plt.axes(projection=ccrs.PlateCarree())

# Add ocean, land, rivers and lakes
ax.add_feature(cfeature.OCEAN.with_scale('50m'))
ax.add_feature(cfeature.LAND.with_scale('50m'))
ax.add_feature(cfeature.RIVERS.with_scale('50m'))
ax.add_feature(cfeature.LAKES.with_scale('50m'))

# # Plot border lines
# for line in borders:
#     ax.plot(line[0::2], line[1::2], '-', lw=1, color='k',
#             transform=ccrs.Geodetic())

# Plot the AOT
LON, LAT = np.meshgrid(lonRect, latRect)
pcmesh = plt.pcolormesh(LON, LAT, dataRect, vmin=vmin, vmax=vmax, transform=ccrs.PlateCarree(), cmap='jet')

# Plot the longitude and latitude ticks
ax.set_xticks(np.linspace(axLonRange[0], axLonRange[1], 5, endpoint=True))
ax.set_yticks(np.linspace(axLatRange[0], axLatRange[1], 5, endpoint=True))
lon_formatter = LongitudeFormatter(number_format='.1f',
                                   degree_symbol='',
                                   dateline_direction_label=True)
lat_formatter = LatitudeFormatter(number_format='.1f',
                                  degree_symbol='')
ax.xaxis.set_major_formatter(lon_formatter)
ax.yaxis.set_major_formatter(lat_formatter)
ax.set_title(getattr(fd.variables[product], 'long_name'))
# Set figure extent
ax.set_ylim(axLatRange)
ax.set_xlim(axLonRange)

# Add colorbar
cb_ax = fig.add_axes([0.93, 0.20, 0.02, 0.65])
cbar = fig.colorbar(
    pcmesh, cax=cb_ax,
    ticks=np.linspace(0, 20, 5, endpoint=True),
    orientation='vertical'
    )
cbar.ax.tick_params(direction='out', labelsize=15, pad=5)
cbar.ax.set_title('km', fontsize=10)

# Show figure
plt.show()